In [6]:
import os
import pandas as pd
import numpy as np

# --- CONFIGURACIÓN Y PATHS ---
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
data_path = os.path.join(base_path, "data", "rutas_excel")
input_file = os.path.join(data_path, "rutas.xlsx")
output_file = os.path.join(data_path, "rutas_limpias.csv")

print(f"Base path: {base_path}")
print(f"Archivo de entrada: {input_file}")
print(f"Archivo de salida: {output_file}")


try:
    df = pd.read_excel(input_file)
    print(f"Archivo cargado correctamente con {len(df)} filas y {len(df.columns)} columnas.")
except Exception as e:
    raise RuntimeError(f"Error al cargar el archivo Excel: {e}")

# --- Debug: primeras filas y columnas ---
print("\n Vista previa del dataset original:")
display(df.head(3))

print("\n Columnas originales:")
print(df.columns.tolist())


df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("á", "a")
    .str.replace("é", "e")
    .str.replace("í", "i")
    .str.replace("ó", "o")
    .str.replace("ú", "u")
)

print("\n Columnas renombradas:")
print(df.columns.tolist())

# Fechas
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Numéricos: forzar conversión segura
numeric_cols = [
    "vehicle_capacity_m3", "volumen_cargado_m3", "carga_pct",
    "n_paradas", "n_paquetes", "cliente_top_count", "paquetes_no_entregados",
    "porcentaje_entregas_exitosas", "duracion_min", "tiempo_conduccion_min",
    "tiempo_paradas_min", "tiempo_espera_deposito_min", "tiempo_carga_min",
    "km_total", "desviacion_km", "velocidad_media_kmh",
    "n_incidentes", "costo_est_operativo"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Cadenas: limpiar espacios y forzar tipo string
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()


print("\n Validación inicial de datos:")

# --- Duplicados ---
duplicates = df.duplicated(subset=["route_id", "date"]).sum()
print(f"• Duplicados detectados: {duplicates}")

# --- Nulos ---
null_counts = df.isna().sum()
print(f"\n• Valores nulos por columna:")
print(null_counts[null_counts > 0])

# --- Rango esperado: porcentaje de carga y éxito ---
if "carga_pct" in df.columns:
    outliers_carga = df[~df["carga_pct"].between(0, 100, inclusive="both")]
    print(f"\n Filas con carga_pct fuera de rango (0-100): {len(outliers_carga)}")

if "porcentaje_entregas_exitosas" in df.columns:
    outliers_entregas = df[~df["porcentaje_entregas_exitosas"].between(0, 100, inclusive="both")]
    print(f"Filas con porcentaje_entregas_exitosas fuera de rango (0-100): {len(outliers_entregas)}")


if duplicates > 0:
    df = df.drop_duplicates(subset=["route_id", "date"], keep="first")

# Corregir valores fuera de rango
if "carga_pct" in df.columns:
    df.loc[df["carga_pct"] > 100, "carga_pct"] = 100
    df.loc[df["carga_pct"] < 0, "carga_pct"] = 0

if "porcentaje_entregas_exitosas" in df.columns:
    df.loc[df["porcentaje_entregas_exitosas"] > 100, "porcentaje_entregas_exitosas"] = 100
    df.loc[df["porcentaje_entregas_exitosas"] < 0, "porcentaje_entregas_exitosas"] = 0

# Rellenar nulos clave si es razonable
df["weekday"] = df["weekday"].fillna(df["date"].dt.weekday if "date" in df.columns else np.nan)
df["turno"] = df["turno"].fillna("no_definido")



print("\n✅ Validación posterior a limpieza:")
print(f"Filas finales: {len(df)}")
print(f"Columnas: {len(df.columns)}")

print("\n🧩 Tipos de datos finales:")
print(df.dtypes)

print("\n📊 Estadísticas resumidas:")
display(df.describe(include="all").T.head(10))

try:
    df.to_csv(output_file, index=False)
    print(f"\n Archivo limpio exportado correctamente a: {output_file}")
except Exception as e:
    print(f" Error al exportar CSV: {e}")



print("\n📋 RESUMEN DE CALIDAD DE DATOS")
print(f"• Filas totales: {len(df)}")
print(f"• Columnas totales: {len(df.columns)}")
print(f"• Nulos totales: {df.isna().sum().sum()}")
print(f"• Duplicados totales: {df.duplicated().sum()}")

print("\n Ejemplo de filas limpias:")
display(df.sample(5, random_state=42))


📁 Base path: C:\Users\tomyg\OneDrive\Desktop\logistica-analisis
📄 Archivo de entrada: C:\Users\tomyg\OneDrive\Desktop\logistica-analisis\data\rutas_excel\rutas.xlsx
💾 Archivo de salida: C:\Users\tomyg\OneDrive\Desktop\logistica-analisis\data\rutas_excel\rutas_limpias.csv
✅ Archivo cargado correctamente con 500 filas y 35 columnas.

🔍 Vista previa del dataset original:


,route_id,date,weekday,week_number,driver_id,origin_barrio,lat_origin,lon_origin,barrios_recorridos,tipo_vehiculo,vehicle_capacity_m3,volumen_cargado_m3,carga_pct,n_paradas,n_paquetes,cliente_top_count,km_total,desviacion_km,velocidad_media_kmh,duracion_min,tiempo_conduccion_min,tiempo_paradas_min,tiempo_espera_deposito_min,tiempo_carga_min,n_incidentes,motivo_incidente,paquetes_no_entregados,porcentaje_entregas_exitosas,prioridad,entregas_retrasadas,clima,turno,estado_ruta,notas_logisticas,costo_est_operativo
0,2000,2025-01-27,0,4,3,La Plata,-34.922486,-57.952770,La Plata; Tolosa,Furgoneta,8,7.84,98.0,71,81,6,24.6,0.07,17.57,224,84,130,10,16,1,Cliente ausente,1,98.77,media,1,Nublado,mañana,con_incidente,Devolver a depósito,29.01
1,2001,2025-03-25,1,12,1,Abasto,-34.820862,-57.922538,Abasto; Villa Elisa; City Bell,Camioneta,13,4.41,33.9,85,102,7,18.7,0.99,13.68,241,82,159,2,32,0,NaN,0,100.00,alta,0,Despejado,mañana,completa_ok,NaN,30.33
2,2002,2025-01-11,5,1,3,Abasto,-34.817327,-57.916802,Abasto; Lisandro Olmos,Camioneta,13,6.04,46.5,88,101,8,29.4,0.64,19.60,254,90,164,9,34,0,NaN,0,100.00,baja,0,Despejado,mañana,completa_ok,NaN,33.10



📋 Columnas originales:
['route_id', 'date', 'weekday', 'week_number', 'driver_id', 'origin_barrio', 'lat_origin', 'lon_origin', 'barrios_recorridos', 'tipo_vehiculo', 'vehicle_capacity_m3', 'volumen_cargado_m3', 'carga_pct', 'n_paradas', 'n_paquetes', 'cliente_top_count', 'km_total', 'desviacion_km', 'velocidad_media_kmh', 'duracion_min', 'tiempo_conduccion_min', 'tiempo_paradas_min', 'tiempo_espera_deposito_min', 'tiempo_carga_min', 'n_incidentes', 'motivo_incidente', 'paquetes_no_entregados', 'porcentaje_entregas_exitosas', 'prioridad', 'entregas_retrasadas', 'clima', 'turno', 'estado_ruta', 'notas_logisticas', 'costo_est_operativo']

✅ Columnas renombradas:
['route_id', 'date', 'weekday', 'week_number', 'driver_id', 'origin_barrio', 'lat_origin', 'lon_origin', 'barrios_recorridos', 'tipo_vehiculo', 'vehicle_capacity_m3', 'volumen_cargado_m3', 'carga_pct', 'n_paradas', 'n_paquetes', 'cliente_top_count', 'km_total', 'desviacion_km', 'velocidad_media_kmh', 'duracion_min', 'tiempo_cond

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
route_id,500.0,NaN,NaN,NaN,2249.5,2000.0,2124.75,2249.5,2374.25,2499.0,144.481833
date,500,NaN,NaN,NaN,2025-07-01 10:50:52.800000,2025-01-01 00:00:00,2025-03-28 12:00:00,2025-07-04 00:00:00,2025-09-26 06:00:00,2025-12-31 00:00:00,NaN
weekday,500.0,NaN,NaN,NaN,2.922,0.0,1.0,3.0,5.0,6.0,1.995967
week_number,500.0,NaN,NaN,NaN,25.914,0.0,12.0,26.0,38.0,52.0,15.234131
driver_id,500.0,NaN,NaN,NaN,3.926,1.0,2.0,4.0,6.0,7.0,1.976952
origin_barrio,500,18,Ringuelet,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat_origin,500.0,NaN,NaN,NaN,-34.899252,-34.953134,-34.927274,-34.91719,-34.87169,-34.79461,0.040802
lon_origin,500.0,NaN,NaN,NaN,-57.942763,-57.997043,-57.95957,-57.948234,-57.922951,-57.870181,0.030679
barrios_recorridos,500,215,City Bell; Joaquin Gorina,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tipo_vehiculo,500,3,Camioneta,230,NaN,NaN,NaN,NaN,NaN,NaN,NaN



💾 Archivo limpio exportado correctamente a: C:\Users\tomyg\OneDrive\Desktop\logistica-analisis\data\rutas_excel\rutas_limpias.csv

📋 RESUMEN DE CALIDAD DE DATOS
• Filas totales: 500
• Columnas totales: 35
• Nulos totales: 0
• Duplicados totales: 0

🔎 Ejemplo de filas limpias:


,route_id,date,weekday,week_number,driver_id,origin_barrio,lat_origin,lon_origin,barrios_recorridos,tipo_vehiculo,vehicle_capacity_m3,volumen_cargado_m3,carga_pct,n_paradas,n_paquetes,cliente_top_count,km_total,desviacion_km,velocidad_media_kmh,duracion_min,tiempo_conduccion_min,tiempo_paradas_min,tiempo_espera_deposito_min,tiempo_carga_min,n_incidentes,motivo_incidente,paquetes_no_entregados,porcentaje_entregas_exitosas,prioridad,entregas_retrasadas,clima,turno,estado_ruta,notas_logisticas,costo_est_operativo
361,2361,2025-09-10,2,36,5,Berisso,-34.857556,-57.876314,Berisso; La Plata; Tolosa,Furgoneta,8,5.42,67.8,87,103,5,34.9,0.87,16.36,322,128,194,3,19,0,nan,0,100.00,media,0,Nublado,mañana,con_incidente,nan,41.51
73,2073,2025-04-29,1,17,7,Villa Montoro,-34.898341,-57.952763,Villa Montoro; Ringuelet; Gonnet; City Bell,Furgoneta,8,7.84,98.0,79,92,6,28.9,0.00,16.67,303,104,199,0,34,0,nan,0,100.00,media,1,Despejado,mañana,con_incidente,nan,36.63
374,2374,2025-10-14,1,41,7,Los Hornos,-34.919804,-57.988961,Los Hornos; La Plata,Camioneta,13,4.46,34.3,90,104,7,35.4,1.27,17.70,243,120,123,3,26,0,nan,0,100.00,media,0,Tormenta,mañana,completa_ok,nan,32.03
155,2155,2025-07-01,1,26,2,Lisandro Olmos,-34.800641,-57.903338,Lisandro Olmos; Melchor Romero,Furgoneta,8,7.84,98.0,82,97,4,34.5,1.21,18.16,276,114,162,1,29,0,nan,2,97.94,media,1,Despejado,mañana,parcial,Contactar cliente,36.31
104,2104,2025-06-11,2,23,1,La Plata,-34.918424,-57.952990,La Plata; Villa Elisa; City Bell,Furgoneta,8,7.76,97.0,71,88,10,34.9,0.39,10.42,292,201,91,18,22,0,nan,1,98.86,baja,1,Despejado,mañana,parcial,Reintentar mañana,38.70
